# Chapter 6 - Prompt Engineering

Unlike other computer programs, where we interact with source code, LLM interactions is through text. The input text we provide to LLM is called as Prompt. Different LLMs are trained to take different length of input tokens. This is called as context window/length. GPT-2.5 Turbo has a context length of 16K. GPT-4 on the ohter hand can ingest upto 128K tokens. Using this prompt the language models makes an inference. The process of deriving the ouptut based on the inference is called as completion.

A typical prompt is composed of the following elements

* A question or an instruction to the language model
* Context information
* Any other examples

We will begin this chapter with introductions history of prompt and prompt engineering. Prompt engineering is a discipline to help us adapt LLMs for diverse tasks. This is a new paradigm in machine learning. Historically, models were trained to perform a single task restiricted to a single domain. Prompt engineering enables us to use a pre-trained language model adapt itself to perform different tasks. We will intially look at the foundational techniques like zero-shot and few-shot learning. For the purpose of demonstration, we will leverage LangChain PromptTemplates and Ollama models.Post pre-training, during instruction fine tuning phase, models get trained to perform multiple tasks through instruction datasets. Hence they may have different prompting syntax. LangChain provides us with an unified way of interacting with different models.

## Why prompting

Today's software market demands a variety of NLP systems. Starting from grammatical correction, text summariziation, question answering, plaguraism detection, language transalation, sentiment analysis, text classification, and many more. Data collection and labeling for each of these tasks are manual labour intensive. Once a system is developed and deployed, the whole process has to be repeated for creating another system. Even if the other system is not drastically different than the former one. 

A complex model, when trained with a limited dataset is prone to exploit spurious correlations in the training dataset. Consider the case of a large language models. These models are pre-trained with billions of tokens. However while fine tuning for a specific task, the training data is very limited, in order of magnitude much smaller than the initial data used for pre-training.

With a few examples humans can learn new NLP tasks. They don't need to be provided with large amount of traning data. A task speciiced in natrual language, along with a couple of examples is suffiecient.

### In-context learning



In [20]:
from langchain_community.llms import Ollama

MODEL = "phi3:latest"

llm = Ollama(model=MODEL, temperature=0.5)


In [21]:
prompt1 = "Will cotton sink in water"
llm.invoke(prompt1)

"Yes, cotton will generally float on the surface of water. This is because cotton fibers are light and have a low density compared to water. Cotton's ability to absorb water can also increase its overall volume without significantly increasing its mass, which further contributes to it floating rather than sinking when placed in water. However, if enough cotton becomes submerged due to absorption of water and the weight increases beyond a certain point, then at that moment, it would start to sink, but typically this happens after significant soaking."

In [19]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("will {item} sink in water?")
print(llm.invoke(prompt.format(item="cotton")))


Yes, cotton will generally sink in water. Cotton is a natural fiber that is not very dense compared to most materials commonly found in nature or manufactured products. However, it's important to note that the amount of cotton and how it's packed together can affect whether it sinks or floats. If you have a large quantity of loose cotton fibers, they may disperse and float on water due to their low density and surface tension effects. But typically, when considering an individual piece of cotton fabric in still water, it will sink because the overall density is greater than that of water.
Yes, cotton will generally float on the surface of water. Cotton is a natural fiber that has a low density and high volume relative to its mass, which causes it to be less dense than water (density of water is approximately 1 gram per cubic centimeter at room temperature). When you place cotton in water, it will not sink but instead disperse on the surface. However, if enough cotton fabric were compac

### Zero Shot

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_string = "Give me {number} prompt string to generate realistic {article_type} titles."

#
# from_template is a classmethod
# Returns an instance of PromptTemplate
synthetic_prompt = PromptTemplate.from_template(prompt_string, template_format="f-string")

synthetic_prompt

PromptTemplate(input_variables=['article_type', 'number'], template='Give me {number} prompt string to generate realistic {article_type} titles.')

In [13]:
examples = ["Is {name} is a planet. Tell me its {adjective} properties", "Is {fname} is a fruit? Tell me its {fadjective} properties"]
prompt = PromptTemplate.from_examples(examples, suffix="_next_", input_variables=["name","adjective","fname","fadjective"])
prompt

PromptTemplate(input_variables=['adjective', 'fadjective', 'fname', 'name'], template='\n\nIs {name} is a planet. Tell me its {adjective} properties\n\nIs {fname} is a fruit? Tell me its {fadjective} properties\n\n_next_')

## Few Shot

In [27]:
examples =[ 
    {
    "context": 
        """"
        A "whatpu" is a small, furry animal native to Tanzania.
        """,
    "question":
        """
        An example of a sentence that uses the word whatpu is:
        """,
    "answer":
        """
        We were traveling in Africa and we saw these very cute whatpus.
        """
},
    {
    "context": 
        """"
        To do a "farduddle" means to jump up and down really fast.
        """,
    "question":
        """
        An example of a sentence that uses the word farduddle is:
        """,
    "answer":
        """
        When we won the game, we all started to farduddle in celebration.
        """
},  
          ]

In [37]:
from langchain_core.prompts import FewShotPromptTemplate


example_prompt = PromptTemplate(
    input_variables=["context","question","answer"]
   ,template = "context: {context}\n question: {question}\n{answer}"
)


print("********** Single Example ***************")
print(example_prompt.format(**examples[0]))

print("********** Print Complete Prompt ***************")

context="""
    "Bodega" is a slang for small convienience store in new york
    """
question="make an example sentence that uses Bodega"


fewshot_prompt = FewShotPromptTemplate(
    examples = examples
   ,example_prompt = example_prompt
   ,suffix="context: {context}\nquestion:{question}"
   ,input_variables=["context","question"]
)

print(fewshot_prompt.format(context=context, question=question))

********** Single Example ***************
context: "
        A "whatpu" is a small, furry animal native to Tanzania.
        
 question: 
        An example of a sentence that uses the word whatpu is:
        

        We were traveling in Africa and we saw these very cute whatpus.
        
********** Print Complete Prompt ***************
context: "
        A "whatpu" is a small, furry animal native to Tanzania.
        
 question: 
        An example of a sentence that uses the word whatpu is:
        

        We were traveling in Africa and we saw these very cute whatpus.
        

context: "
        To do a "farduddle" means to jump up and down really fast.
        
 question: 
        An example of a sentence that uses the word farduddle is:
        

        When we won the game, we all started to farduddle in celebration.
        

context: 
    "Bodega" is a slang for small convienience store in new york
    
question:make an example sentence that uses Bodega


In [38]:
llm.invoke(fewshot_prompt.format(context=context, question=question))

'After realizing I forgot my wallet, I hurried to the nearest bodega to quickly grab some essentials before heading back home.'

## Algorithimical Optimization of LLM Prompts